# Oxytocin analysis code

In [ ]:
import utility_functions # loading and processing code

# matplotlib stuff
import ipympl
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon # errorbars and standard deviations
%matplotlib ipympl

# data analysis standards
import numpy as np
import pandas as pd
from scipy import signal
import os, re
from pathlib import Path

# csv writing for figure data etc
import csv
import xmltodict

# Data import and cleaning

Bring in the data, do some basic cleaning before sending it to Kilosort etc

In [ ]:
# recording directory
# assume all subdirectories are from the same session and to be concatenated
# recording_dir = Path('Y:\\Sara\\in vivo phys\\open_ephys\\Mudd data\\awake\\oxt-cre_male_2505\\oxt-cre_m2505_2025-03-17_14-25-13')
recording_dir = 'Y:\\Sara\\in vivo phys\\open_ephys\\Mudd data\\awake\\oxt-cre_male_2506\\oxt-cre_m2506_2025-03-20_14-18-47'

# sig, timestamps = utility_functions.open_raw(recording_dir)



In [3]:
from sklearn.decomposition import PCA

In [ ]:
time_window = 30000*60*5

sos = signal.butter(N=2, Wn=10, fs=30000, btype='highpass', output='sos')
filt_sig = signal.sosfiltfilt(sos, sig[:time_window,:], axis=0)

pca = PCA(n_components=1)
pca.fit(filt_sig)

In [20]:
# across-channel artifacts
sig_clean = np.empty_like(filt_sig) # make a copy for subtraction
for channel in np.arange(sig.shape[1]):
    Wnc = pca.components_.copy() # make a copy for the exclusionary projection matrix
    Wcc = Wnc[:,channel].copy() # and the channel-specific reprojection
    Wnc[:,channel] = 0 # exclude channel's contribution
    Ac = np.matmul(filt_sig, Wnc.T)
    ArtMat = np.linalg.lstsq(Ac,filt_sig[:,channel], rcond=None)[0]
    sig_clean[:,channel] = filt_sig[:,channel] - np.matmul(Ac,ArtMat)

In [ ]:
from matplotlib import pyplot as plt
import ipympl
# %matplotlib ipympl

channels = [10,20,30,40]
fig,ax = plt.subplots(nrows=2, sharex=True)

ax[0].plot(timestamps[:time_window],sig[:time_window,channels])
ax[1].plot(timestamps[:time_window],sig_clean[:,channels])

# ax[0].set_xticks([0])
ax[1].set_xlabel('Time (s)')
ax[0].set_ylabel('Voltage (uV)')
ax[1].set_ylabel('Voltage (uV)')

In [30]:
for ax_ in ax:
    for spine in ax_.spines:
        ax_.spines[spine].set_visible(False)

In [31]:
ax[0].set_title('Raw Recording')
ax[1].set_title('Cleaned Recording')

Text(0.5, 1.0, 'Cleaned Recording')

In [33]:
os.listdir('Y:')

['.DS_Store',
 'Figure_3.png',
 'Isabelle',
 'Janet',
 'Old lab members',
 'Sara',
 'shared (striatum) (Z) - Shortcut.lnk',
 'VS120_pre2023']

In [ ]:
# dir(session.recordnodes[0].recordings[0])
# dir(session)
# dir(session.recordnodes[0].recordings[0].continuous[0])
session.recordnodes[0].recordings[0].continuous[0].metadata

In [ ]:
# from glob import glob


settings_file = glob(f'{recording_dir}{path.sep}**{path.sep}*settings.xml',recursive=True)

if len(settings_file)!=1:
    print('Cannot find a single settings file')


with open(settings_file[0],'r', encoding='utf8') as fid:
#     # settings = fid.readlines()
    settings = xmltodict.parse(fid.read())['SETTINGS']
# settings = xmltodict.parse(settings_file[0])

In [6]:
# from sklearn.decomposition import PCA

utility_functions.ERAASR(sig[0])
# sig

MemoryError: Unable to allocate 75.6 GiB for an array with shape (64, 158575634) and data type float64